<a href="https://www.kaggle.com/code/bryan194/ae-pretrain-gnn-attn-cnn?scriptVersionId=114284532" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### training scheme
#### (1) train denoising auto encoder model using all data including train and test data
#### (2) from the weights of denoising auto encoder model, finetune to predict targets such as reactivity

### rough network architecture
#### inputs -> conv1ds -> aggregation of neighborhoods -> multi head attention -> aggregation of neighborhoods -> multi head attention -> conv1d -> predict
#### this architecture was inspired by https://www.kaggle.com/cpmpml/graph-transfomer


In [1]:
pretrain_dir = None # model dir for resuming training. if None, train from scrach

one_fold = False # if True, train model at only first fold. use if you try a new idea quickly.
run_test = False # if True, use small data. you can check whether this code run or not
denoise = True # if True, use train data whose signal_to_noise > 1

ae_epochs = 20 # epoch of training of denoising auto encoder
ae_epochs_each = 5 # epoch of training of denoising auto encoder each time. 
                   # I use train data (seqlen = 107) and private test data (seqlen = 130) for auto encoder training.
                   # I dont know how to easily fit keras model to use both of different shape data simultaneously, 
                   # so I call fit function several times. 
ae_batch_size = 32

epochs_list = [30, 10, 3, 3, 5, 5]
batch_size_list = [8, 16, 32, 64, 128, 256] 

## copy pretrain model to working dir
import shutil
import glob
if pretrain_dir is not None:
    for d in glob.glob(pretrain_dir + "*"):
        shutil.copy(d, ".")
    
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
import os
import matplotlib.pyplot as plt
%matplotlib inline

## load

In [2]:
import json
import glob
from tqdm.notebook import tqdm

train = pd.read_json("/kaggle/input/stanford-covid-vaccine/train.json",lines=True)
if denoise:
    train = train[train.signal_to_noise > 1].reset_index(drop = True)
test  = pd.read_json("/kaggle/input/stanford-covid-vaccine/test.json",lines=True)
test_pub = test[test["seq_length"] == 107]
test_pri = test[test["seq_length"] == 130]
sub = pd.read_csv("/kaggle/input/stanford-covid-vaccine/sample_submission.csv")

if run_test: ## to test 
    train = train[:30]
    test_pub = test_pub[:30]
    test_pri = test_pri[:30]

As = []
for id in tqdm(train["id"]):
    a = np.load(f"/kaggle/input/stanford-covid-vaccine/bpps/{id}.npy")
    As.append(a)
As = np.array(As)
As_pub = []
for id in tqdm(test_pub["id"]):
    a = np.load(f"/kaggle/input/stanford-covid-vaccine/bpps/{id}.npy")
    As_pub.append(a)
As_pub = np.array(As_pub)
As_pri = []
for id in tqdm(test_pri["id"]):
    a = np.load(f"/kaggle/input/stanford-covid-vaccine/bpps/{id}.npy")
    As_pri.append(a)
As_pri = np.array(As_pri)

In [3]:
print(train.shape)
train.head()

(2096, 19)


,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
1,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."
2,5,id_00ab2d761,GGAAAGCGCCGCGGCGGUAGCGGCAGCGAGGAGCGCUACCAAGGCA...,.....(.(((((.(((((((((...........)))))))..(((....,EEEEESISSSSSISSSSSSSSSHHHHHHHHHHHSSSSSSSMMSSSH...,4.136,1,107,68,"[0.1942, 0.2041, 0.1626, 0.1213, 0.10590000000...","[0.2726, 0.2984, 0.21660000000000001, 0.1637, ...","[0.3393, 0.2728, 0.2005, 0.1703, 0.1495, 0.134...","[0.165, 0.20520000000000002, 0.179, 0.1333, 0....","[0.2864, 0.24710000000000001, 0.2222, 0.1903, ...","[0.7642, 1.6641, 1.0622, 0.5008, 0.4107, 0.133...","[0.9559000000000001, 1.9442, 1.0114, 0.5105000...","[1.9554, 2.1298, 1.0403, 0.609, 0.5486, 0.386,...","[0.22460000000000002, 1.7281, 1.381, 0.6623, 0...","[0.5882000000000001, 1.1786, 0.9704, 0.6035, 0..."
3,6,id_00abef1d7,GGAAAACAAUUGCAUCGUUAGUACGACUCCACAGCGUAAGCUGUGG...,.........((((((((......((((((((((((....)))))))...,EEEEEEEEESSSSSSSSIIIIIISSSSSSSSSSSSHHHHSSSSSSS...,2.485,1,107,68,"[0.422, 0.5478000000000001, 0.4749000000000000...","[0.4801, 0.7943, 0.42160000000000003, 0.397300...","[0.9822000000000001, 1.272, 0.6940000000000001...","[0.5827, 0.7555000000000001, 0.5949, 0.4511, 0...","[0.9306000000000001, 1.0496, 0.5844, 0.7796000...","[0.895, 2.3377, 2.2305, 2.003, 1.9006, 1.0373,...","[0.46040000000000003, 3.6695, 0.78550000000000...","[2.7711, 7.365, 1.6924000000000001, 1.43840000...","[1.073, 2.8604000000000003, 1.9936, 1.0273, 1....","[2.0964, 3.3688000000000002, 0.6399, 2.1053, 1..."
4,7,id_00b436dec,GGAAAUCAUCGAGGACGGGUCCGUUCAGCACGCGAAAGCGUCGUGA...,.....(((((((((((..(((((((((..((((....))))..)))...,EEEEESSSSSSSSSSSIISSSSSSSSSIISSSSHHHHSSSSIISSS...,1.727,1,107,68,"[0.4843, 0.5233, 0.4554, 0.43520000000000003, ...","[0.8719, 1.0307, 0.6649, 0.34500000000000003, ...","[0.7045, 0.7775000000000001, 0.5662, 0.4561, 0...","[0.384, 0.723, 0.4766, 0.30260000000000004, 0....","[0.7429, 0.9137000000000001, 0.480400000000000...","[1.1576, 1.5137, 1.3382, 1.5622, 1.2121, 0.295...","[1.6912, 5.2652, 2.3901, 0.45890000000000003, ...","[1.8641, 2.3767, 1.149, 1.0132, 0.9876, 0.0, 0...","[0.49060000000000004, 4.6339, 1.95860000000000...","[1.2852000000000001, 2.5460000000000003, 0.234..."


In [4]:
print(test.shape)
test.head()

(3634, 7)


,index,id,sequence,structure,predicted_loop_type,seq_length,seq_scored
0,0,id_00073f8be,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68
1,1,id_000ae4237,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91
2,2,id_00131c573,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68
3,3,id_00181fd34,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68
4,4,id_0020473f7,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91


In [5]:
print(sub.shape)
sub.head()

(457953, 6)


,id_seqpos,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,id_00073f8be_0,0.0,0.0,0.0,0.0,0.0
1,id_00073f8be_1,0.0,0.0,0.0,0.0,0.0
2,id_00073f8be_2,0.0,0.0,0.0,0.0,0.0
3,id_00073f8be_3,0.0,0.0,0.0,0.0,0.0
4,id_00073f8be_4,0.0,0.0,0.0,0.0,0.0


## target

In [6]:
targets = list(sub.columns[1:])
print(targets)

y_train = []
seq_len = train["seq_length"].iloc[0]
seq_len_target = train["seq_scored"].iloc[0]
ignore = -10000
ignore_length = seq_len - seq_len_target
for target in targets:
    y = np.vstack(train[target])
    dummy = np.zeros([y.shape[0], ignore_length]) + ignore
    y = np.hstack([y, dummy])
    y_train.append(y)
y = np.stack(y_train, axis = 2)
y.shape

['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']


(2096, 107, 5)

## structure adj

In [7]:
def get_structure_adj(train):
    ## get adjacent matrix from structure sequence
    
    ## here I calculate adjacent matrix of each base pair, 
    ## but eventually ignore difference of base pair and integrate into one matrix
    Ss = []
    for i in tqdm(range(len(train))):
        seq_length = train["seq_length"].iloc[i]
        structure = train["structure"].iloc[i]
        sequence = train["sequence"].iloc[i]

        cue = []
        a_structures = {
            ("A", "U") : np.zeros([seq_length, seq_length]),
            ("C", "G") : np.zeros([seq_length, seq_length]),
            ("U", "G") : np.zeros([seq_length, seq_length]),
            ("U", "A") : np.zeros([seq_length, seq_length]),
            ("G", "C") : np.zeros([seq_length, seq_length]),
            ("G", "U") : np.zeros([seq_length, seq_length]),
        }
        a_structure = np.zeros([seq_length, seq_length])
        for i in range(seq_length):
            if structure[i] == "(":
                cue.append(i)
            elif structure[i] == ")":
                start = cue.pop()
#                 a_structure[start, i] = 1
#                 a_structure[i, start] = 1
                a_structures[(sequence[start], sequence[i])][start, i] = 1
                a_structures[(sequence[i], sequence[start])][i, start] = 1
        
        a_strc = np.stack([a for a in a_structures.values()], axis = 2)
        a_strc = np.sum(a_strc, axis = 2, keepdims = True)
        Ss.append(a_strc)
    
    Ss = np.array(Ss)
    print(Ss.shape)
    return Ss
Ss = get_structure_adj(train)
Ss_pub = get_structure_adj(test_pub)
Ss_pri = get_structure_adj(test_pri)


(2096, 107, 107, 1)



(629, 107, 107, 1)



(3005, 130, 130, 1)


## distance adj

In [8]:
def get_distance_matrix(As):
    ## adjacent matrix based on distance on the sequence
    ## D[i, j] = 1 / (abs(i - j) + 1) ** pow, pow = 1, 2, 4
    
    idx = np.arange(As.shape[1])
    Ds = []
    for i in range(len(idx)):
        d = np.abs(idx[i] - idx)
        Ds.append(d)

    Ds = np.array(Ds) + 1
    Ds = 1/Ds
    Ds = Ds[None, :,:]
    Ds = np.repeat(Ds, len(As), axis = 0)
    
    Dss = []
    for i in [1, 2, 4]: 
        Dss.append(Ds ** i)
    Ds = np.stack(Dss, axis = 3)
    print(Ds.shape)
    return Ds

Ds = get_distance_matrix(As)
Ds_pub = get_distance_matrix(As_pub)
Ds_pri = get_distance_matrix(As_pri)

(2096, 107, 107, 3)
(629, 107, 107, 3)
(3005, 130, 130, 3)


In [9]:
## concat adjecent
As = np.concatenate([As[:,:,:,None], Ss, Ds], axis = 3).astype(np.float32)
As_pub = np.concatenate([As_pub[:,:,:,None], Ss_pub, Ds_pub], axis = 3).astype(np.float32)
As_pri = np.concatenate([As_pri[:,:,:,None], Ss_pri, Ds_pri], axis = 3).astype(np.float32)
del Ss, Ds, Ss_pub, Ds_pub, Ss_pri, Ds_pri
As.shape, As_pub.shape, As_pri.shape

((2096, 107, 107, 5), (629, 107, 107, 5), (3005, 130, 130, 5))

## node

In [10]:
## sequence
def return_ohe(n, i):
    tmp = [0] * n
    tmp[i] = 1
    return tmp

def get_input(train):
    ## get node features, which is one hot encoded
    mapping = {}
    vocab = ["A", "G", "C", "U"]
    for i, s in enumerate(vocab):
        mapping[s] = return_ohe(len(vocab), i)
    X_node = np.stack(train["sequence"].apply(lambda x : list(map(lambda y : mapping[y], list(x)))))

    mapping = {}
    vocab = ["S", "M", "I", "B", "H", "E", "X"]
    for i, s in enumerate(vocab):
        mapping[s] = return_ohe(len(vocab), i)
    X_loop = np.stack(train["predicted_loop_type"].apply(lambda x : list(map(lambda y : mapping[y], list(x)))))
    
    mapping = {}
    vocab = [".", "(", ")"]
    for i, s in enumerate(vocab):
        mapping[s] = return_ohe(len(vocab), i)
    X_structure = np.stack(train["structure"].apply(lambda x : list(map(lambda y : mapping[y], list(x)))))
    
    
    X_node = np.concatenate([X_node, X_loop], axis = 2)
    
    ## interaction
    a = np.sum(X_node * (2 ** np.arange(X_node.shape[2])[None, None, :]), axis = 2)
    vocab = sorted(set(a.flatten()))
    print(vocab)
    ohes = []
    for v in vocab:
        ohes.append(a == v)
    ohes = np.stack(ohes, axis = 2)
    X_node = np.concatenate([X_node, ohes], axis = 2).astype(np.float32)
    
    
    print(X_node.shape)
    return X_node

X_node = get_input(train)
X_node_pub = get_input(test_pub)
X_node_pri = get_input(test_pri)

[17, 18, 20, 24, 33, 34, 36, 40, 65, 66, 68, 72, 129, 130, 132, 136, 257, 258, 260, 264, 513, 514, 516, 520, 1025, 1026, 1028, 1032]
(2096, 107, 39)
[17, 18, 20, 24, 33, 34, 36, 40, 65, 66, 68, 72, 129, 130, 132, 136, 257, 258, 260, 264, 513, 514, 516, 520, 1025, 1026, 1028, 1032]
(629, 107, 39)
[17, 18, 20, 24, 33, 34, 36, 40, 65, 66, 68, 72, 129, 130, 132, 136, 257, 258, 260, 264, 513, 514, 516, 520, 1025, 1026, 1028, 1032]
(3005, 130, 39)


## model

In [11]:
import tensorflow as tf
from tensorflow.keras import layers as L
import tensorflow_addons as tfa
from tensorflow.keras import backend as K

def mcrmse(t, p, seq_len_target = seq_len_target):
    ## calculate mcrmse score by using numpy
    t = t[:, :seq_len_target]
    p = p[:, :seq_len_target]
    
    score = np.mean(np.sqrt(np.mean(np.mean((p - t) ** 2, axis = 1), axis = 0)))
    return score

def mcrmse_loss(t, y, seq_len_target = seq_len_target):
    ## calculate mcrmse score by using tf
    t = t[:, :seq_len_target]
    y = y[:, :seq_len_target]
    
    loss = tf.reduce_mean(tf.sqrt(tf.reduce_mean(tf.reduce_mean((t - y) ** 2, axis = 1), axis = 0)))
    return loss

def attention(x_inner, x_outer, n_factor, dropout):
    x_Q =  L.Conv1D(n_factor, 1, activation='linear', 
                  kernel_initializer='glorot_uniform',
                  bias_initializer='glorot_uniform',
                 )(x_inner)
    x_K =  L.Conv1D(n_factor, 1, activation='linear', 
                  kernel_initializer='glorot_uniform',
                  bias_initializer='glorot_uniform',
                 )(x_outer)
    x_V =  L.Conv1D(n_factor, 1, activation='linear', 
                  kernel_initializer='glorot_uniform',
                  bias_initializer='glorot_uniform',
                 )(x_outer)
    x_KT = L.Permute((2, 1))(x_K)
    res = L.Lambda(lambda c: K.batch_dot(c[0], c[1]) / np.sqrt(n_factor))([x_Q, x_KT])
#     res = tf.expand_dims(res, axis = 3)
#     res = L.Conv2D(16, 3, 1, padding = "same", activation = "relu")(res)
#     res = L.Conv2D(1, 3, 1, padding = "same", activation = "relu")(res)
#     res = tf.squeeze(res, axis = 3)
    att = L.Lambda(lambda c: K.softmax(c, axis=-1))(res)
    att = L.Lambda(lambda c: K.batch_dot(c[0], c[1]))([att, x_V])
    return att

def multi_head_attention(x, y, n_factor, n_head, dropout):
    if n_head == 1:
        att = attention(x, y, n_factor, dropout)
    else:
        n_factor_head = n_factor // n_head
        heads = [attention(x, y, n_factor_head, dropout) for i in range(n_head)]
        att = L.Concatenate()(heads)
        att = L.Dense(n_factor, 
                      kernel_initializer='glorot_uniform',
                      bias_initializer='glorot_uniform',
                     )(att)
    x = L.Add()([x, att])
    x = L.LayerNormalization()(x)
    if dropout > 0:
        x = L.Dropout(dropout)(x)
    return x

def res(x, unit, kernel = 3, rate = 0.1):
    h = L.Conv1D(unit, kernel, 1, padding = "same", activation = None)(x)
    h = L.LayerNormalization()(h)
    h = L.LeakyReLU()(h)
    h = L.Dropout(rate)(h)
    return L.Add()([x, h])

def forward(x, unit, kernel = 3, rate = 0.1):
#     h = L.Dense(unit, None)(x)
    h = L.Conv1D(unit, kernel, 1, padding = "same", activation = None)(x)
    h = L.LayerNormalization()(h)
    h = L.Dropout(rate)(h)
#         h = tf.keras.activations.swish(h)
    h = L.LeakyReLU()(h)
    h = res(h, unit, kernel, rate)
    return h

def adj_attn(x, adj, unit, n = 2, rate = 0.1):
    x_a = x
    x_as = []
    for i in range(n):
        x_a = forward(x_a, unit)
        x_a = tf.matmul(adj, x_a) ## aggregate neighborhoods
        x_as.append(x_a)
    if n == 1:
        x_a = x_as[0]
    else:
        x_a = L.Concatenate()(x_as)
    x_a = forward(x_a, unit)
    return x_a


def get_base(config):
    ## base model architecture 
    ## node, adj -> middle feature
    
    node = tf.keras.Input(shape = (None, X_node.shape[2]), name = "node")
    adj = tf.keras.Input(shape = (None, None, As.shape[3]), name = "adj")
    
    adj_learned = L.Dense(1, "relu")(adj)
    adj_all = L.Concatenate(axis = 3)([adj, adj_learned])
        
    xs = []
    xs.append(node)
    x1 = forward(node, 128, kernel = 3, rate = 0.0)
    x2 = forward(x1, 64, kernel = 6, rate = 0.0)
    x3 = forward(x2, 32, kernel = 15, rate = 0.0)
    x4 = forward(x3, 16, kernel = 30, rate = 0.0)
    x = L.Concatenate()([x1, x2, x3, x4])
    
    for unit in [64, 32]:
        x_as = []
        for i in range(adj_all.shape[3]):
            x_a = adj_attn(x, adj_all[:, :, :, i], unit, rate = 0.0)
            x_as.append(x_a)
        x_c = forward(x, unit, kernel = 30)
        
        x = L.Concatenate()(x_as + [x_c])
        x = forward(x, unit)
        x = multi_head_attention(x, x, unit, 4, 0.0)
        xs.append(x)
        
    x = L.Concatenate()(xs)

    model = tf.keras.Model(inputs = [node, adj], outputs = [x])
    return model


def get_ae_model(base, config):
    ## denoising auto encoder part
    ## node, adj -> middle feature -> node
    
    node = tf.keras.Input(shape = (None, X_node.shape[2]), name = "node")
    adj = tf.keras.Input(shape = (None, None, As.shape[3]), name = "adj")

    x = base([L.SpatialDropout1D(0.3)(node), adj])
    x = forward(x, 64, rate = 0.3)
    p = L.Dense(X_node.shape[2], "sigmoid")(x)
    
    loss = - tf.reduce_mean(20 * node * tf.math.log(p + 1e-4) + (1 - node) * tf.math.log(1 - p + 1e-4))
    model = tf.keras.Model(inputs = [node, adj], outputs = [loss])
    
    opt = get_optimizer()
    model.compile(optimizer = opt, loss = lambda t, y : y)
    return model


def get_model(base, config):
    ## regression part
    ## node, adj -> middle feature -> prediction of targets
    
    node = tf.keras.Input(shape = (None, X_node.shape[2]), name = "node")
    adj = tf.keras.Input(shape = (None, None, As.shape[3]), name = "adj")
    
    x = base([node, adj])
    x = forward(x, 128, rate = 0.4)
    x = L.Dense(5, None)(x)

    model = tf.keras.Model(inputs = [node, adj], outputs = [x])
    
    opt = get_optimizer()
    model.compile(optimizer = opt, loss = mcrmse_loss, metrics=['accuracy'])
    return model

def get_optimizer():
#     sgd = tf.keras.optimizers.SGD(0.05, momentum = 0.9, nesterov=True)
    adam = tf.optimizers.Adam()
#     radam = tfa.optimizers.RectifiedAdam()
#     lookahead = tfa.optimizers.Lookahead(adam, sync_period=6)
#     swa = tfa.optimizers.SWA(adam)
    return adam

/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


## pretrain

In [12]:
## here train denoising auto encoder model using all data

config = {} ## not use now
if ae_epochs > 0:
    base = get_base(config)
    ae_model = get_ae_model(base, config)
    ## TODO : simultaneous train
    for i in range(ae_epochs//ae_epochs_each):
        print(f"------ {i} ------")
        print("--- train ---")
        ae_model.fit([X_node, As], [X_node[:,0]],
                  epochs = ae_epochs_each,
                  batch_size = ae_batch_size)
        print("--- public ---")
        ae_model.fit([X_node_pub, As_pub], [X_node_pub[:,0]],
                  epochs = ae_epochs_each,
                  batch_size = ae_batch_size)
        print("--- private ---")
        ae_model.fit([X_node_pri, As_pri], [X_node_pri[:,0]],
                  epochs = ae_epochs_each,
                  batch_size = ae_batch_size)
        gc.collect()
    print("****** save ae model ******")
    base.save_weights("./base_ae")

------ 0 ------
--- train ---
Epoch 1/5
66/66 [==============================] - 6s 89ms/step - loss: 0.9170
Epoch 2/5
66/66 [==============================] - 5s 83ms/step - loss: 0.3265
Epoch 3/5
66/66 [==============================] - 5s 79ms/step - loss: 0.1673
Epoch 4/5
66/66 [==============================] - 5s 81ms/step - loss: 0.1134
Epoch 5/5
66/66 [==============================] - 5s 80ms/step - loss: 0.0836
--- public ---
Epoch 1/5
20/20 [==============================] - 2s 116ms/step - loss: 0.0678
Epoch 2/5
20/20 [==============================] - 2s 77ms/step - loss: 0.0688
Epoch 3/5
20/20 [==============================] - 2s 81ms/step - loss: 0.0630
Epoch 4/5
20/20 [==============================] - 2s 77ms/step - loss: 0.0633
Epoch 5/5
20/20 [==============================] - 2s 77ms/step - loss: 0.0567
--- private ---
Epoch 1/5
94/94 [==============================] - 10s 103ms/step - loss: 0.0563
Epoch 2/5
94/94 [==============================] - 9s 94ms/step - l

## train

In [ ]:
## here train regression model from pretrain auto encoder model

from sklearn.model_selection import KFold
kfold = KFold(5, shuffle = True, random_state = 42)

scores = []
train_loss = []
val_loss = []
train_acc = []
val_acc = []

preds = np.zeros([len(X_node), X_node.shape[1], 5])
for i, (tr_idx, va_idx) in enumerate(kfold.split(X_node, As)):
    print(f"------ fold {i} start -----")
    print(f"------ fold {i} start -----")
    print(f"------ fold {i} start -----")
    X_node_tr = X_node[tr_idx]
    X_node_va = X_node[va_idx]
    As_tr = As[tr_idx]
    As_va = As[va_idx]
    y_tr = y[tr_idx]
    y_va = y[va_idx]
    
    base = get_base(config)
    if ae_epochs > 0:
        print("****** load ae model ******")
        base.load_weights("./base_ae")
    model = get_model(base, config)
    if pretrain_dir is not None:
        d = f"./model{i}"
        print(f"--- load from {d} ---")
        model.load_weights(d)
    for epochs, batch_size in zip(epochs_list, batch_size_list):
        print(f"epochs : {epochs}, batch_size : {batch_size}")
        history = model.fit([X_node_tr, As_tr], [y_tr],
                  validation_data=([X_node_va, As_va], [y_va]),
                  epochs = epochs,
                  batch_size = batch_size, validation_freq = 1)
        
        train_loss += [round(l, 4) for l in history.history['loss']]
        val_loss += [round(l, 4) for l in history.history['val_loss']]
        train_acc += [round(l, 4) for l in history.history['accuracy']]
        val_acc += [round(l, 4) for l in history.history['val_accuracy']]

        
    model.save_weights(f"./model{i}")
    p = model.predict([X_node_va, As_va])
    scores.append(mcrmse(y_va, p))
    print(f"fold {i}: mcrmse {scores[-1]}")
    preds[va_idx] = p
    if one_fold:
        break
        
pd.to_pickle(preds, "oof.pkl")

------ fold 0 start -----
------ fold 0 start -----
------ fold 0 start -----
****** load ae model ******
epochs : 30, batch_size : 8
Epoch 1/30
210/210 [==============================] - 17s 81ms/step - loss: 0.5438 - val_loss: 0.3228
Epoch 2/30
210/210 [==============================] - 11s 51ms/step - loss: 0.3351 - val_loss: 0.3005
Epoch 3/30
210/210 [==============================] - 10s 50ms/step - loss: 0.3057 - val_loss: 0.2860
Epoch 4/30
210/210 [==============================] - 11s 51ms/step - loss: 0.2868 - val_loss: 0.2675
Epoch 5/30
210/210 [==============================] - 11s 50ms/step - loss: 0.2787 - val_loss: 0.2679
Epoch 6/30
210/210 [==============================] - 11s 51ms/step - loss: 0.2697 - val_loss: 0.2590
Epoch 7/30
210/210 [==============================] - 11s 53ms/step - loss: 0.2642 - val_loss: 0.2593
Epoch 8/30
210/210 [==============================] - 11s 51ms/step - loss: 0.2609 - val_loss: 0.2544
Epoch 9/30
210/210 [==============================

In [ ]:
print(scores)
print('train_loss', train_loss)
print('val_loss', val_loss)
print('train_acc', train_acc)
print('val_acc', val_acc)
train_loss = [0.5505, 0.3342, 0.306, 0.2898, 0.2789, 0.2712, 0.2647, 0.2607, 0.2559, 0.2511, 0.2506, 0.2437, 0.2407, 0.2381, 0.2355, 0.2363, 0.2308, 0.2271, 0.2258, 0.2236, 0.2219, 0.2204, 0.2188, 0.217, 0.215, 0.2135, 0.2128, 0.2112, 0.2098, 0.2084, 0.2023, 0.1993, 0.1983, 0.1981, 0.1957, 0.1962, 0.1951, 0.1946, 0.1941, 0.1934, 0.1903, 0.1885, 0.1874, 0.1864, 0.1852, 0.1839, 0.1838, 0.1829, 0.1823, 0.1823, 0.1821, 0.1818, 0.1815, 0.1811, 0.1808, 0.1806, 0.5515, 0.3396, 0.3069, 0.2893, 0.2796, 0.2727, 0.2664, 0.261, 0.2572, 0.2533, 0.2471, 0.245, 0.2431, 0.2388, 0.237, 0.2343, 0.231, 0.2309, 0.2294, 0.2253, 0.2237, 0.2233, 0.2201, 0.2188, 0.2172, 0.2166, 0.2142, 0.2126, 0.2109, 0.2114, 0.2046, 0.2014, 0.2011, 0.1999, 0.1987, 0.1987, 0.1978, 0.1971, 0.1962, 0.1962, 0.1935, 0.1905, 0.1896, 0.1886, 0.1874, 0.1861, 0.1854, 0.1853, 0.1848, 0.1847, 0.1844, 0.1844, 0.184, 0.1835, 0.1829, 0.1826, 0.5469, 0.3354, 0.3046, 0.2872, 0.2763, 0.2674, 0.2625, 0.2576, 0.2537, 0.2488, 0.2441, 0.2421, 0.2387, 0.2368, 0.2341, 0.2312, 0.2288, 0.2258, 0.2242, 0.224, 0.2217, 0.2197, 0.2167, 0.2158, 0.2147, 0.2135, 0.2118, 0.2107, 0.2086, 0.2064, 0.2016, 0.1976, 0.1975, 0.1968, 0.1963, 0.1951, 0.1948, 0.1937, 0.1934, 0.1935, 0.1903, 0.1883, 0.187, 0.1857, 0.1846, 0.1836, 0.1833, 0.1829, 0.1821, 0.1819, 0.1821, 0.1823, 0.1813, 0.181, 0.1806, 0.1804, 0.5532, 0.3303, 0.3001, 0.2863, 0.2769, 0.2701, 0.2638, 0.259, 0.254, 0.251, 0.2463, 0.2432, 0.2401, 0.2363, 0.2336, 0.2309, 0.2294, 0.2289, 0.2243, 0.2237, 0.2209, 0.2194, 0.2182, 0.2147, 0.2132, 0.2132, 0.2108, 0.209, 0.2082, 0.2081, 0.2026, 0.1987, 0.1977, 0.1972, 0.1958, 0.1955, 0.195, 0.1947, 0.1948, 0.193, 0.19, 0.1876, 0.1876, 0.1866, 0.1848, 0.1841, 0.1833, 0.1834, 0.1824, 0.183, 0.1823, 0.1819, 0.1817, 0.1813, 0.181, 0.181, 0.5815, 0.344, 0.3127, 0.2918, 0.2808, 0.2734, 0.2672, 0.2612, 0.2574, 0.2525, 0.2489, 0.2452, 0.2432, 0.2381, 0.2369, 0.2341, 0.2325, 0.2288, 0.2275, 0.2239, 0.2232, 0.2217, 0.2193, 0.2182, 0.2165, 0.2154, 0.2138, 0.2116, 0.2111, 0.2085, 0.204, 0.1997, 0.1985, 0.198, 0.1975, 0.1963, 0.1967, 0.1948, 0.1942, 0.1949, 0.192, 0.189, 0.1886, 0.1871, 0.1858, 0.185, 0.1843, 0.1836, 0.1833, 0.183, 0.1833, 0.1825, 0.1824, 0.1822, 0.1817, 0.1816]
val_loss = [0.3299, 0.3068, 0.2893, 0.2676, 0.2731, 0.2575, 0.2604, 0.2585, 0.2507, 0.2533, 0.2453, 0.2456, 0.2448, 0.2442, 0.241, 0.2481, 0.2389, 0.2406, 0.2401, 0.2482, 0.2376, 0.2355, 0.2372, 0.241, 0.235, 0.2343, 0.2333, 0.2373, 0.2376, 0.2372, 0.2334, 0.2323, 0.2331, 0.2307, 0.233, 0.2319, 0.2323, 0.2328, 0.2312, 0.2318, 0.2299, 0.2303, 0.2321, 0.23, 0.23, 0.2303, 0.2306, 0.23, 0.23, 0.2302, 0.2301, 0.2303, 0.23, 0.2304, 0.23, 0.2298, 0.3237, 0.2968, 0.2742, 0.2653, 0.2633, 0.2663, 0.2546, 0.2468, 0.2444, 0.2414, 0.2428, 0.24, 0.2391, 0.2407, 0.2385, 0.2336, 0.2336, 0.2365, 0.2344, 0.2372, 0.2343, 0.2299, 0.231, 0.2365, 0.2296, 0.2309, 0.2282, 0.2279, 0.2263, 0.2258, 0.2239, 0.2254, 0.2286, 0.2239, 0.2225, 0.2253, 0.2247, 0.2239, 0.2238, 0.227, 0.223, 0.2218, 0.2228, 0.2217, 0.2215, 0.2206, 0.2209, 0.2212, 0.2214, 0.2217, 0.221, 0.2214, 0.2214, 0.2216, 0.2209, 0.2209, 0.3368, 0.3088, 0.288, 0.2768, 0.2706, 0.2646, 0.2631, 0.2638, 0.2634, 0.2562, 0.2523, 0.2487, 0.25, 0.2463, 0.2503, 0.2488, 0.2479, 0.2486, 0.2462, 0.2488, 0.2449, 0.2396, 0.2402, 0.2426, 0.2492, 0.239, 0.2405, 0.2393, 0.2392, 0.2411, 0.236, 0.237, 0.2367, 0.2373, 0.2389, 0.2356, 0.2363, 0.2352, 0.2391, 0.2378, 0.2348, 0.2353, 0.2346, 0.2347, 0.2356, 0.2348, 0.2356, 0.2357, 0.2352, 0.2356, 0.2356, 0.2358, 0.2355, 0.2352, 0.2351, 0.2353, 0.3342, 0.2976, 0.2829, 0.2766, 0.2708, 0.2703, 0.269, 0.2654, 0.258, 0.2613, 0.252, 0.2523, 0.2515, 0.2505, 0.2534, 0.2491, 0.258, 0.2482, 0.2483, 0.2495, 0.2489, 0.245, 0.2436, 0.246, 0.2463, 0.2428, 0.2438, 0.2442, 0.2405, 0.2423, 0.2403, 0.2386, 0.2384, 0.2395, 0.2382, 0.2383, 0.2385, 0.2405, 0.2389, 0.2377, 0.2385, 0.2372, 0.2378, 0.2381, 0.2375, 0.237, 0.238, 0.2394, 0.2374, 0.2386, 0.2378, 0.238, 0.2377, 0.2378, 0.2381, 0.2383, 0.3388, 0.3083, 0.2789, 0.2765, 0.2655, 0.2592, 0.2566, 0.252, 0.257, 0.2488, 0.2526, 0.2451, 0.2473, 0.245, 0.252, 0.241, 0.24, 0.2377, 0.2388, 0.2385, 0.2393, 0.2356, 0.237, 0.2383, 0.2347, 0.2347, 0.2346, 0.2358, 0.234, 0.2319, 0.2311, 0.2305, 0.2307, 0.2316, 0.2317, 0.2304, 0.2307, 0.2301, 0.2325, 0.2312, 0.2299, 0.2337, 0.2309, 0.2303, 0.2296, 0.2294, 0.2299, 0.2302, 0.2299, 0.23, 0.2304, 0.2308, 0.2307, 0.2305, 0.2309, 0.231]
# fig = px.line(
#     history.history, y=['loss', 'val_loss'],
#     labels={'index': 'epoch', 'value': 'MCRMSE'}, 
#     title='Training History')
# fig.show()

plt.plot(range(sum(epochs_list)), train_loss, val_loss)



## predict

In [ ]:
p_pub = 0
p_pri = 0
for i in range(5):
    model.load_weights(f"./model{i}")
    p_pub += model.predict([X_node_pub, As_pub]) / 5
    p_pri += model.predict([X_node_pri, As_pri]) / 5
    if one_fold:
        p_pub *= 5
        p_pri *= 5
        break

for i, target in enumerate(targets):
    test_pub[target] = [list(p_pub[k, :, i]) for k in range(p_pub.shape[0])]
    test_pri[target] = [list(p_pri[k, :, i]) for k in range(p_pri.shape[0])]

## sub

In [ ]:
preds_ls = []
for df, preds in [(test_pub, p_pub), (test_pri, p_pri)]:
    for i, uid in enumerate(df.id):
        single_pred = preds[i]

        single_df = pd.DataFrame(single_pred, columns=targets)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]

        preds_ls.append(single_df)

preds_df = pd.concat(preds_ls)
preds_df.to_csv("submission.csv", index = False)
preds_df.head()

In [ ]:
print(scores)
print(np.mean(scores))